Daniel Aguilar
Homework3
CSP 228
Prof. James Liporace
04/14/2025

**Instructions**  
Progrmming Projects

Chapter 6 

6.2,6.3,6.4

Chapter 7 

7.1,.7.2,7.3,.7.4



In [4]:
"""
Problem 6.2:
Every positive integer can be divided into a set of integer factors.
The minimum set of factors must be a collection of prime numbers,
where a prime number is one that is only evenly divisible by 1 and itself.

Write a recursive function, factor(), that returns the list of integer
factors of x.


1. Only test factors between 2 and the square root of x (use lowest up to sqrt(x)).
2. Add an optional parameter to factor(x, lowest) for the lowest possible integer factor.
3. The recursive function should:
   a. Check base cases (0, 1, and negatives).
   b. If lowest evenly divides x (x % lowest == 0):
        - Add lowest to the factors of x divided by lowest.
   c. If lowest does NOT evenly divide x:
        - Look for factors with the next higher possible factor (lowest+1).
   d. A factor can appear more than once in the final list.
4. For negative x, return the factors of the positive version but with factor 1 replaced by -1.
5. Test on:
   -- Compound (nonprime) numbers
   -- Prime numbers
   -- Special cases of 0 and 1
   -- Negative integers

Use Chapter 5's LinkedList structure (plus __iter__ to traverse it).
"""

class Link:
    #One datum in a linked list.
    def __init__(self, data, next=None):
        self.__data = data
        self.__next = next

    def getData(self):
        return self.__data

    def getNext(self):
        return self.__next

    def setNext(self, link):
        if link is None or isinstance(link, Link):
            self.__next = link
        else:
            raise Exception("Next link must be Link or None")

    def __str__(self):
        return str(self.getData())


class LinkedList:
    #Minimal Chapter 5 singly linked list.
    def __init__(self):
        self.__first = None

    def getFirst(self):
        return self.__first

    def setFirst(self, link):
        if link is None or isinstance(link, Link):
            self.__first = link
        else:
            raise Exception("First link must be Link or None")

    def isEmpty(self):
        return self.getFirst() is None

    def insert(self, datum):
        #Insert new datum at front of list (used to build factor lists).
        new_link = Link(datum, self.getFirst())
        self.setFirst(new_link)

    def __iter__(self):
        #Allow iteration: for d in linked_list.
        link = self.getFirst()
        while link is not None:
            yield link.getData()
            link = link.getNext()

    def __str__(self):
        pieces = [str(d) for d in self]
        return "[" + " > ".join(pieces) + "]"


def factor(x, lowest=2):
    """
    Recursive factor() per Problem 6.2:

    - Base cases:
        * x == 0  → special case of 0               # “special cases of 0 and 1”
        * x == 1  → special case of 1
        * x == -1 → handle negative 1

    - Negative integers:
        * Prepend -1 then factor(-x, lowest)
          # “factors of the positive version but with the factor 1 replaced by -1”

    - Prime detection:
        * If lowest*lowest > x, x is prime → insert(x)
          # “only the factors between 2 and the square root of x need to be tested”

    - Divide step:
        * If x % lowest == 0:
            -- Recursively factor(x//lowest, lowest)
            -- Then insert(lowest)
          # “If it does, then add lowest to the factors of x divided by lowest.”

    - Skip step:
        * Else → factor(x, lowest+1)
          # “If lowest doesn't evenly divide x, then look for factors with the next higher possible factor.”

    - A factor can appear > once in final list.
    """
    result = LinkedList()

    # ---- Base cases of 0, 1, and -1 ----
    if x == 0:
        result.insert(0)
        return result
    if x == 1:
        result.insert(1)
        return result
    if x == -1:
        result.insert(-1)
        return result

    # ---- Negative integer handling ----
    if x < 0:
        # factors of positive version but with factor 1 replaced by −1
        result.insert(-1)
        tail = factor(-x, lowest)
        for d in tail:          # traverse the positive factors
            result.insert(d)     # prepend so final order is correct
        return result

    # ---- Prime detection (lowest > sqrt(x)) ----
    if lowest * lowest > x:
        result.insert(x)  # x is prime, minimum set of primes includes x itself
        return result

    # ---- Divide step (x % lowest == 0) ----
    if x % lowest == 0:
        tail = factor(x // lowest, lowest)
        for d in tail:      # build up factors of (x//lowest)
            result.insert(d)
        result.insert(lowest)
        return result

    # ---- Skip to next higher integer factor ----
    return factor(x, lowest + 1)


# ---------------- Demo / Output ----------------
if __name__ == "__main__":
    # Test special cases of 0 and 1
    print("factor(0)  # special case 0 →", factor(0))
    print("factor(1)  # special case 1 →", factor(1))

    # Test prime numbers (minimum set of prime factors is itself)
    print("factor(2)  # prime number (evenly divisible only by 1 and itself) →", factor(2))
    print("factor(13) # prime number →", factor(13))

    # Test compound (nonprime) numbers
    print("factor(12) # factors of 12 → minimum set of primes →", factor(12))
    print("factor(60) # factors of 60 →", factor(60))

    # Test negative integer handling
    print("factor(-45) # negative integer → factors of 45 with -1 replacing 1 →", factor(-45))

factor(0)  # special case 0 → [0]
factor(1)  # special case 1 → [1]
factor(2)  # prime number (evenly divisible only by 1 and itself) → [2]
factor(13) # prime number → [13]
factor(12) # factors of 12 → minimum set of primes → [2 > 3 > 2]
factor(60) # factors of 60 → [2 > 3 > 5 > 2]
factor(-45) # negative integer → factors of 45 with -1 replacing 1 → [3 > 5 > 3 > -1]


In [5]:
"""
Problem 6.3:
Implement the recursive approach to raising a number to a power,
as described in the “Raising a Number to a Power” section near the end
of this chapter. Write the recursive power() function. For extra utility,
make use of this transformation to handle negative integer exponents:
    x^y = 1 / x^(-y) when y < 0

Test your function on several combinations including positive and
negative integers and the special cases where the exponent is 0 and 1
(x^0 = 1 and x^1 = x).
"""

def power(x, y):
    """
    Recursive power() per Problem 6.3:

    - Special cases:
        * exponent == 0  → return 1          # “x^0 = 1”
        * exponent == 1  → return x          # “x^1 = x”
    - Negative exponent:
        * y < 0 → use 1 / x^(-y)             # “x^y = 1 / x^-y when y < 0”
    - Recursive (y > 1):
        * x^y = x * x^(y-1)
    """
    # ---- Base cases ----
    if y == 0:
        # “x^0 = 1”
        return 1
    if y == 1:
        # “x^1 = x”
        return x

    # ---- Handle negative integer exponents ----
    if y < 0:
        # “x^y = 1 / x^-y when y < 0”
        return 1 / power(x, -y)

    # ---- Recursive step for positive y > 1 ----
    # “x^y = x * x^(y-1)”
    return x * power(x, y - 1)


# ---------------- Demo / Output ----------------
if __name__ == "__main__":
    # Positive exponent tests
    print("power(2, 0)   # special case x^0=1  →", power(2, 0))
    print("power(5, 1)   # special case x^1=x  →", power(5, 1))
    print("power(3, 3)   # 3^3 = 27            →", power(3, 3))

    # Larger positive exponent
    print("power(2, 10)  # 2^10 = 1024         →", power(2, 10))

    # Negative exponent tests
    print("power(2, -1)  # 2^-1 = 1/2          →", power(2, -1))
    print("power(5, -3)  # 5^-3 = 1/125        →", power(5, -3))

    # Mixed sign
    print("power(-2, 3)  # (-2)^3 = -8         →", power(-2, 3))
    print("power(-2, 4)  # (-2)^4 = 16         →", power(-2, 4))

power(2, 0)   # special case x^0=1  → 1
power(5, 1)   # special case x^1=x  → 5
power(3, 3)   # 3^3 = 27            → 27
power(2, 10)  # 2^10 = 1024         → 1024
power(2, -1)  # 2^-1 = 1/2          → 0.5
power(5, -3)  # 5^-3 = 1/125        → 0.008
power(-2, 3)  # (-2)^3 = -8         → -8
power(-2, 4)  # (-2)^4 = 16         → 16


In [6]:
"""
Problem 6.4: The Knapsack Problem

Write a program that solves the knapsack problem for an arbitrary
knapsack capacity and series of weights. Assume the weights are stored
in an array. The recursive knapsack() function’s arguments are:

  • target: the remaining weight to fill
  • weights: the list of item weights
  • index: the current position in the weights array
  • current: a LinkedList of the items selected so far

When the sum of selected items equals the original target, the function
prints the list of weights in the knapsack.
"""

# ----------------------------------------------------------------------
# Chapter 5 Linked-List Implementation
# ----------------------------------------------------------------------

class Link(object):  # One datum in a linked list
    def __init__(self, datum, next=None):          # Constructor Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        self.__data = datum
        self.__next = next

    def getData(self):                             # Return stored datum
        return self.__data

    def setData(self, datum):                      # Change stored datum
        self.__data = datum

    def getNext(self):                             # Return next Link
        return self.__next

    def setNext(self, link):                       # Change next Link Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        if link is None or isinstance(link, Link):
            self.__next = link
        else:
            raise Exception("Next link must be Link or None")

    def isLast(self):                              # Test if this is the last Link
        return self.getNext() is None

    def __str__(self):                             # String representation of Link
        return str(self.getData())


def identity(x): return x                        # Helper key function Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)


class LinkedList(object):  # A linked list of data elements

    def __init__(self):                             # Constructor Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        self.__first = None

    # Basic accessors and tests
    def getFirst(self): return self.__first         # Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
    def setFirst(self, link):                       # Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        if link is None or isinstance(link, Link):
            self.__first = link
        else:
            raise Exception("First link must be Link or None")

    def getNext(self): return self.getFirst()       # Synonym for getFirst Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
    def setNext(self, link): self.setFirst(link)    # Synonym for setFirst Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)

    def isEmpty(self):                              # Test for empty list Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        return self.getFirst() is None

    def first(self):                                # Return first item, not Link Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        if self.isEmpty():
            raise Exception("No first item in empty list")
        return self.getFirst().getData()

    # Traversal and utility methods
    def traverse(self, func=print):                  # Apply func to each item ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        link = self.getFirst()
        while link is not None:
            func(link.getData())
            link = link.getNext()

    def __len__(self):                               # Enable len(self) ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        l, link = 0, self.getFirst()
        while link is not None:
            l += 1
            link = link.getNext()
        return l

    def __str__(self):                               # String form: [a > b > …] ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        result, link = "[", self.getFirst()
        while link is not None:
            if len(result) > 1: result += " > "
            result += str(link)
            link = link.getNext()
        return result + "]"

    # Insertion and search
    def insert(self, datum):                         # Insert at start ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        link = Link(datum, self.getFirst())
        self.setFirst(link)

    def find(self, goal, key=identity):              # Return Link whose data-key == goal ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        link = self.getFirst()
        while link is not None:
            if key(link.getData()) == goal:
                return link
            link = link.getNext()

    def search(self, goal, key=identity):            # Return datum for first match ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        link = self.find(goal, key)
        return link.getData() if link is not None else None

    def insertAfter(self, goal, newDatum, key=identity):  # Insert after matching link ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        link = self.find(goal, key)
        if link is None: return False
        newLink = Link(newDatum, link.getNext())
        link.setNext(newLink)
        return True

    # Deletion
    def deleteFirst(self):                           # Delete and return first datum ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        if self.isEmpty():
            raise Exception("Cannot delete first of empty list")
        first = self.getFirst()
        self.setFirst(first.getNext())
        return first.getData()

    def delete(self, goal, key=identity):            # Delete first matching datum ‡Chapter 5 from Data Structures  Algorithms in Python_2022_ (Z-Library).pdf](file-service://file-QvZorXgAqV4DMd2C1kadPS)
        if self.isEmpty():
            raise Exception("Cannot delete from empty linked list")
        previous = self
        while previous.getNext() is not None:
            link = previous.getNext()
            if key(link.getData()) == goal:
                previous.setNext(link.getNext())
                return link.getData()
            previous = link
        raise Exception("No item with matching key found in list")


# ----------------------------------------------------------------------
# Recursive Knapsack Solver
# ----------------------------------------------------------------------

def knapsack(target, weights, index=0, current=None):
    """
    Recursively find all combinations of weights[index:] that sum to target.
    When target == 0, prints the current LinkedList of selected weights.
    """
    if current is None:
        current = LinkedList()

    # Base case: exact fit
    if target == 0:
        print(current)       # Each solution printed as a linked list
        return

    # No more items or target negative: dead end
    if index >= len(weights) or target < 0:
        return

    # 1) Include weights[index]
    #    Create a new list copying current and insert this weight
    incl = LinkedList()
    # Copy all items from current into incl
    def _copier(x, lst=incl): lst.insert(x)
    current.traverse(lambda x: incl.insert(x))
    incl.insert(weights[index])

    # Recurse with reduced target and next index
    knapsack(target - weights[index], weights, index + 1, incl)

    # 2) Exclude weights[index], move on
    knapsack(target, weights, index + 1, current)


# ----------------------------------------------------------------------
# Example Usage
# ----------------------------------------------------------------------

if __name__ == "__main__":
    W = [11, 8, 7, 6, 5]
    T = 20
    print(f"Knapsack combinations for target = {T} and weights = {W}:")
    knapsack(T, W)

Knapsack combinations for target = 20 and weights = [11, 8, 7, 6, 5]:
[5 > 8 > 7]
